In [37]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
import glob 
import json 
import shutil
path = "/home/imaginglyceum/Desktop/reu_suren_lab2024/ex_obj-det/SARscope/"

In [38]:
from pylabel import importer
def convert_coco2Yolo(source_path,dst_path):
    if not os.path.exists(dst_path):
        os.mkdir(dst_path)
    json_file=glob.glob(source_path+"/*.json")
    dataset=importer.ImportCoco(json_file[0], path_to_images=source_path, name="SARscope")
    dataset.export.ExportToYoloV5(dst_path)

In [39]:
source_path=path + "train"
dst_path=path+ "train/labels"
convert_coco2Yolo(source_path,dst_path)

source_path=path + "test"
dst_path=path+ "test/labels"
convert_coco2Yolo(source_path,dst_path)

source_path=path + "valid"
dst_path=path+ "valid/labels"
convert_coco2Yolo(source_path,dst_path)



Exporting YOLO files...: 100%|█████████████████████| 1346/1346 [00:03<00:00, 365.82it/s]


In [48]:
config_file_template = f'''
train: {path}train_images
val: {path}valid_images
test: {path}test_images

nc: 2
names: ['background','ship']
'''.format(path = path)

with open(f"{path}/train/dataset.yaml".format(path = path), 'w') as f:
    f.write(config_file_template)

In [49]:
import yaml 
with open(f"{path}/train/dataset.yaml".format(path = path), 'r') as f:
    file_path=yaml.safe_load(f)
print(file_path)

{'train': '/home/imaginglyceum/Desktop/reu_suren_lab2024/ex_obj-det/SARscope/train_images', 'val': '/home/imaginglyceum/Desktop/reu_suren_lab2024/ex_obj-det/SARscope/valid_images', 'test': '/home/imaginglyceum/Desktop/reu_suren_lab2024/ex_obj-det/SARscope/test_images', 'nc': 2, 'names': ['background', 'ship']}


In [50]:
from ultralytics import YOLO
model = YOLO('yolov8n.yaml').load('yolov8n.pt')

Transferred 355/355 items from pretrained weights


In [54]:
model.train(data=f'{path}train/dataset.yaml'.format(path=path),epochs=15,imgsz=640,device=0)

Ultralytics YOLOv8.2.31 🚀 Python-3.11.7 torch-2.3.0 CUDA:0 (NVIDIA GeForce RTX 4090, 24209MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/home/imaginglyceum/Desktop/reu_suren_lab2024/ex_obj-det/SARscope/train/dataset.yaml, epochs=15, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train14, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_c

train: Scanning /home/imaginglyceum/Desktop/reu_suren_lab2024/ex_obj-det/SARscope/train_

WARNING ⚠️ No labels found in /home/imaginglyceum/Desktop/reu_suren_lab2024/ex_obj-det/SARscope/train_images.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.



val: Scanning /home/imaginglyceum/Desktop/reu_suren_lab2024/ex_obj-det/SARscope/valid_im

WARNING ⚠️ No labels found in /home/imaginglyceum/Desktop/reu_suren_lab2024/ex_obj-det/SARscope/valid_images.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.


...

       1/15      2.71G          0       0.97          0          0        640: 100%|████
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)

                   all       1346          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, can not compute metrics without labels



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       2/15      2.36G          0      0.103          0          0        640: 100%|████
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)


RuntimeError: torch.cat(): expected a non-empty list of Tensors

In [ ]:
model.export(path)

In [ ]:
from PIL import Image
test_images=glob.glob(f"{path}/test/*.jpg".format(path=path))
results=model(test_images[4],stream=False)

res=results[0].plot()
Image.fromarray(res)

In [ ]:
test_model=YOLO("/kaggle/working/runs/detect/train/weights/best.pt")
results=test_model(test_images[:4],stream=False)
for i in range(4):
    res=results[i].plot()
    display(Image.fromarray(res))